<API Naver 통합 검색어 트렌드>  https://datalab.naver.com/에서의 검색어트랜드에서 가져온 데이터 

In [ ]:
# 0번) 준비
import requests, json

In [ ]:
#1번) key 등록
CLIENT_ID = "s54DLwetu6NTphXQpHxC"
CLIENT_SECRET = "_Iud1BL7ky"

In [ ]:
# naver 예시코드(request.urlopen 예) ==> 예시이니 동과
import os
import sys
import urllib.request
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"
url = "https://openapi.naver.com/v1/datalab/search";
body = "{\"startDate\":\"2017-01-01\",\"endDate\":\"2017-04-30\",\"timeUnit\":\"month\",\"keywordGroups\":[{\"groupName\":\"한글\",\"keywords\":[\"한글\",\"korean\"]},{\"groupName\":\"영어\",\"keywords\":[\"영어\",\"english\"]}],\"device\":\"pc\",\"ages\":[\"1\",\"2\"],\"gender\":\"f\"}";

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

In [ ]:
# 2번) name header 올리기
headers = { "Content-Type": "application/json",
            "X-Naver-Client-Id": CLIENT_ID,
            "X-Naver-Client-Secret": CLIENT_SECRET }

In [ ]:
# 3번) naver 검색순위  선생님(requests 사용)  --검색어트렌드에 입력하는 것과 같음
url = "https://openapi.naver.com/v1/datalab/search"
params = {
"startDate": "2023-01-01",
"endDate": "2023-12-31",
"timeUnit": "month",
"keywordGroups": [
{"groupName": "트위터", "keywords": ["트위터", "트윗"]},
{"groupName": "페이스북", "keywords": ["페이스북", "페북"]},
{"groupName": "인스타그램", "keywords": ["인스타그램", 
                                    "인스타"]}]}
response = requests.post(url, data = json.dumps(params), headers=headers)

In [ ]:
# 4) 확인(결과 보고 beautifulSoup로 파써할지 json인지 확인함)

In [ ]:
response.content

In [ ]:
response.json()

In [ ]:
datas = response.json()['results']
datas

<API Kakao 지도 서비스>

In [ ]:
# kakao 지도
restapi_key = "bd88efd8afb2711bbd519fd5cdaeb4e1"
url = "https://dapi.kakao.com/v2/local/search/address.json"
params = { "query": '집현동로 77' }
headers = {"Authorization": "KakaoAK " + restapi_key}    #KakaoAK 위에 한칸 띄우기
response = requests.get(url, headers=headers, params=params)
response

In [ ]:
datas = response.json()
datas['documents'][0]

<직방 이용하여 API 구현하기> -geohash로 데이터검색

In [ ]:
def search_location(query):
    restapi_key ='bd88efd8afb2711bbd519fd5cdaeb4e1'
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    params = { "query": query }
    headers = {"Authorization": "KakaoAK " + restapi_key}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()['documents'][0]
    else:
        return response.status_code

In [ ]:
address = search_location('중동')
lng = float(address['x'])
lat = float(address['y'])
print(lng,lat)

In [ ]:
address = search_location('집현동로 77')
print(address['address_name'])
print(address['x'])
print(address['y'])

In [ ]:
!pip install geohash2

In [ ]:
import geohash2

In [ ]:
geohash = geohash2.encode(lat, lng, precision=5)
geohash

In [ ]:
#직방 geohash 검색
url = f'https://apis.zigbang.com/v2/items/oneroom?geohash={geohash}\
&depositMin=0&rentMin=0&salesTypes[0]=월세&salesTypes[1]=전세\
&domain=zigbang&checkAnyItemWithoutFilter=true'
response = requests.get(url)
datas = response.json()['items']
ids = [data["itemId"] for data in datas]
len(ids), ids[:5]

In [ ]:
url = "https://apis.zigbang.com/v2/items/list"
params = {"item_ids":"ids", "domain": "zigbang"}
response = requests.post(url, params)
datas = response.json()['items']
items_id

함수1) 카카오 주소 검색 > 경도, 위도

In [ ]:
def search_location(query):
    restapi_key = '60b71cac95c11ebfeb2ee9d26ca44bbc'
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    params = { "query": query }
    headers = {"Authorization": "KakaoAK " + restapi_key}
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        try:
            address = response.json()['documents'][0]
            return float(address['x']), float(address['y'])
        except:
            return '검색안됨'
    else:
        return response.status_code

함수2) 직방 > geohash > ID를 가져오는 api

In [ ]:
def search_ids(geohash):
    url = f'https://apis.zigbang.com/v2/items/oneroom'

    params = {'geohash': geohash,
              'depositMin':'0',
              'rentMin':'0',
              'salesTypes' : ['월세', '전세'],
              'domain':'zigbang',
              'checkAnyItemWithoutFilter':'true'}

    response = requests.get(url, params)
    if response.status_code == 200:
        datas = response.json()['items']
        ids = [data["itemId"] for data in datas]
        return ids
    else:
        return response.status_code

함수3) ID > 원룸 정보를 가져오는 api

In [ ]:
def search_oneroom(ids):
    url = "https://apis.zigbang.com/v2/items/list"
    params = {"item_ids": id, "domain": "zigbang"}
    response = requests.post(url, params)
    result = response.json()
    final_result = []
    for item in result['items']:
        sales_type = item['sales_type']
        deposit = item['deposit']
        rent = item['rent']
        area = item['전용면적']['m2']
        address = item['address']
        final_result.append([sales_type, deposit, rent, area, address])
    return final_result

In [ ]:
## 파일을 저장하는 함수 

In [ ]:
# 나
def write_txt(result):
    with open('result.csv','w',encoding='euc-kr') as f:
#    item=['월세', 1000, 70, 20,82, '서초구 우면동']  한개만
        for item in result:    
            item=list(map(str, item))
            item=',', join(item)
        #    f.writelines(item)   == 한줄로 띄어쓰기 등 안됨
            f.write(item)
            f.write('\n')
    print('파일 생성 완료')

In [ ]:
#선생님
def write_txt(file, result, mode='w'):
    with open(file, mode, encoding='euc-kr') as f:
        for item in result: # item = ['월세', 1000, 70, 20.82, '서초구 우면동']
            item = list(map(str, item))
            item = ', '.join(item)
            f.write(item)
            f.write('\n')
    print('파일 생성 완료')

In [ ]:
## 만들어진 파일을 db에 저장하기 

In [64]:
import pymysql

In [ ]:
def insert_maria(result):
    conn = pymysql.connect(user = 'maria',
                            password = '1234',
                            host = 'localhost',
                            port = 3306,
                            database = 'python_db'
                            )
    cursor = conn.cursor()
    sql = '''insert into oneroom values (%s, %s, %s, %s, %s, %s)'''
    for index, item in enumerate(result):
        try:
            cursor.execute(sql, item)
        except:
            continue
        if index % 10 == 0:
            conn.commit()
    conn.commit()
    conn.close()